<a href="https://colab.research.google.com/github/anoushkagarg003/contract_analysis_workflow/blob/main/workflow_law_agreement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
pip install -q google-generativeai langgraph pydantic

Note: you may need to restart the kernel to use updated packages.


In [29]:
import google.generativeai as genai
from typing import List, Optional
from pydantic import BaseModel
from langgraph.graph import StateGraph, END

In [30]:
from langchain_community.chat_models import ChatOllama
model = ChatOllama(model="llama3") 

In [31]:
class Section(BaseModel):
    content: str
    tone: Optional[str] = None
    bias: Optional[str] = None
    suggestion: Optional[str] = None

class AuditorState(BaseModel):
    document: str
    sections: Optional[List[Section]] = []

In [32]:
# Tool 1: Split Document
def split_document(state: AuditorState) -> AuditorState:
    prompts = f"""
    Break this legal/business document into logical sections.
    Respond with a JSON list of strings, where each string is a meaningful section.

    Document:
    {state.document}
    """
    response = model.invoke(prompts).content  # ✅ Use `.content` for ChatOllama
    try:
        import json
        content = response.strip().strip("```json").strip("```")
        sections = json.loads(content)
    except:
        sections = [state.document]  # fallback: no splitting
    return AuditorState(document=state.document, sections=[Section(content=s) for s in sections])


# Tool 2: Analyze Tone
def analyze_tone(state: AuditorState) -> AuditorState:
    updated_sections = []
    for section in state.sections:
        prompt = f"Analyze the tone of this section:\n\n{section.content}\n\nRespond with 1-2 words (e.g., 'formal', 'aggressive')."
        tone = model.invoke(prompt).content.strip()
        section.tone = tone
        updated_sections.append(section)
    state.sections = updated_sections
    return state


# Tool 3: Detect Bias
def check_bias(state: AuditorState) -> AuditorState:
    updated_sections = []
    for section in state.sections:
        prompt = f"Does the following section contain any bias? Answer clearly like 'Yes, it's biased towards X' or 'No bias detected'.\n\n{section.content}"
        bias = model.invoke(prompt).content.strip()
        section.bias = bias
        updated_sections.append(section)
    state.sections = updated_sections
    return state


# Tool 4: Suggest Rewrite
def suggest_rewrite(state: AuditorState) -> AuditorState:
    updated_sections = []
    for section in state.sections:
        if section.tone and "aggressive" in section.tone.lower() or "biased" in (section.bias or "").lower():
            prompt = f"Rewrite this section to make it more neutral and professional:\n\n{section.content}"
            suggestion = model.invoke(prompt).content.strip()
            section.suggestion = suggestion
        updated_sections.append(section)
    state.sections = updated_sections
    return state


In [33]:
workflow = StateGraph(AuditorState)
workflow.add_node("split", split_document)
workflow.add_node("tone", analyze_tone)
workflow.add_node("bias", check_bias)
workflow.add_node("rewrite", suggest_rewrite)

workflow.set_entry_point("split")
workflow.add_edge("split", "tone")
workflow.add_edge("tone", "bias")
workflow.add_edge("bias", "rewrite")
workflow.add_edge("rewrite", END)

graph = workflow.compile()

In [34]:
doc = """
This Agreement is irrevocable and shall not be modified under any circumstance by the Buyer.
The Seller reserves the right to terminate at any time without cause.
"""

input_state = AuditorState(document=doc)
final_state = graph.invoke(input_state)

In [35]:
for i, section in enumerate(final_state['sections']):
    print(f"\n--- Section {i+1} ---")
    print(f"Original: {section.content}")
    print(f"Tone: {section.tone}")
    print(f"Bias: {section.bias}")
    if section.suggestion:
        print(f"Suggested Rewrite: {section.suggestion}")


--- Section 1 ---
Original: 
This Agreement is irrevocable and shall not be modified under any circumstance by the Buyer.
The Seller reserves the right to terminate at any time without cause.

Tone: Formal.
Bias: Yes, this section contains bias in favor of the Seller. The language suggests that the Seller has more power and control over the agreement than the Buyer, as it explicitly states that the Buyer cannot modify the agreement under any circumstances, while the Seller can unilaterally terminate the agreement at will. This imbalance favors the Seller's interests over those of the Buyer.
